# ML Zoomcamp 2023, Homework 6 (trees)

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction import DictVectorizer  
from sklearn.metrics import mean_squared_error
from sklearn.tree import export_text
import xgboost as xgb

In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv')
data = data[(data['ocean_proximity'] == 'INLAND') | (data['ocean_proximity'] == '<1H OCEAN')]
data['ocean_proximity'] = data['ocean_proximity'].str.replace('<1H OCEAN','l1H OCEAN')

In [3]:
data.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
701,-121.97,37.64,32.0,1283.0,194.0,485.0,171.0,6.0574,431000.0,l1H OCEAN
830,-121.99,37.61,9.0,3666.0,711.0,2341.0,703.0,4.6458,217000.0,l1H OCEAN
859,-121.97,37.57,21.0,4342.0,783.0,2172.0,789.0,4.6146,247600.0,l1H OCEAN
860,-121.96,37.58,15.0,3575.0,597.0,1777.0,559.0,5.7192,283500.0,l1H OCEAN
861,-121.98,37.58,20.0,4126.0,1031.0,2079.0,975.0,3.6832,216900.0,l1H OCEAN


In [4]:
SEED = 1
data.fillna(0, inplace = True)
data['median_house_value'] = np.log1p(data['median_house_value'])
X_full_train, X_test = train_test_split(data, test_size=0.2, random_state=SEED)
X_train, X_val = train_test_split(X_full_train, test_size=0.25, random_state=SEED)


y_train = X_train.median_house_value.values
y_val = X_val.median_house_value.values
y_test = X_test.median_house_value.values

del X_train['median_house_value']
del X_val['median_house_value']
del X_test['median_house_value']


train_dict = X_train.to_dict(orient='records')
dv = DictVectorizer(sparse=True)
X_train = dv.fit_transform(train_dict)
val_dict = X_val.to_dict(orient='records')
X_val = dv.transform(val_dict) 

In [5]:
tree_reg = DecisionTreeRegressor(max_depth=1)
tree_reg.fit(X_train,y_train)

DecisionTreeRegressor(max_depth=1)

In [6]:
print(export_text(tree_reg, feature_names=dv.get_feature_names()))

|--- ocean_proximity=l1H OCEAN <= 0.50
|   |--- value: [11.61]
|--- ocean_proximity=l1H OCEAN >  0.50
|   |--- value: [12.30]



/Users/anyanordskova/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [7]:
def forest_reg(n):
    forest_reg = RandomForestRegressor(n_estimators=n,random_state=SEED,n_jobs=-1)
    forest_reg.fit(X_train,y_train)

    y_pred = forest_reg.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred,squared=False)
    print(f'RMSE on validation with {n} estimators:', rmse)

for n in range(10,201,10):
    forest_reg(n)

RMSE on validation with 10 estimators: 0.24472569628319227
RMSE on validation with 20 estimators: 0.23864941880469268
RMSE on validation with 30 estimators: 0.2368245014222163
RMSE on validation with 40 estimators: 0.23533546507949907
RMSE on validation with 50 estimators: 0.23492471461649822
RMSE on validation with 60 estimators: 0.23445574076657003
RMSE on validation with 70 estimators: 0.23427611297628062
RMSE on validation with 80 estimators: 0.2344513601594545
RMSE on validation with 90 estimators: 0.23430323364714226
RMSE on validation with 100 estimators: 0.2343039603598108
RMSE on validation with 110 estimators: 0.23413888306600833
RMSE on validation with 120 estimators: 0.23387748792519467
RMSE on validation with 130 estimators: 0.23373630293003372
RMSE on validation with 140 estimators: 0.23357916796133255
RMSE on validation with 150 estimators: 0.23339076413457252
RMSE on validation with 160 estimators: 0.23330937163025903
RMSE on validation with 170 estimators: 0.2332655269

I would say that RMSE stops improving after 170, but the closest option in the answers is 160. 

In [8]:
depths = [10, 15, 20, 25]
def forest_reg(n,d):
    forest_reg = RandomForestRegressor(n_estimators=n,max_depth=d,random_state=SEED,n_jobs=-1)
    forest_reg.fit(X_train,y_train)

    y_pred = forest_reg.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred,squared=False)
    return rmse 


for d in depths:
    rmses = []
    print(f'Depth {d}:')
    for n in range(10,201,10):
        rmses.append(forest_reg(n,d)) 
    print(f'Best RMSE with this depth: {min(rmses)}')

Depth 10:
Best RMSE with this depth: 0.2443480645888683
Depth 15:
Best RMSE with this depth: 0.23442527859375117
Depth 20:
Best RMSE with this depth: 0.2336337692209961
Depth 25:
Best RMSE with this depth: 0.23344717701727608


It seems that 25 is the best number of estimators. 

In [9]:
forest_reg = RandomForestRegressor(n_estimators=10,max_depth=20,random_state=SEED,n_jobs=-1)
forest_reg.fit(X_train,y_train)

for n, f in zip(dv.get_feature_names(),forest_reg.feature_importances_): 
    print(n,f)

households 0.01531441146535209
housing_median_age 0.029918350615801715
latitude 0.1023788857263761
longitude 0.08588294549904486
median_income 0.33582315293094067
ocean_proximity=INLAND 0.18437086435737374
ocean_proximity=l1H OCEAN 0.1819085440416149
population 0.027965370050039177
total_bedrooms 0.01615743731895244
total_rooms 0.02028003799450423


/Users/anyanordskova/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Hence we see that median_income is the most important feature. 

In [10]:
features = dv.get_feature_names()
dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=features)
dval = xgb.DMatrix(X_val, label=y_val, feature_names=features)

In [11]:
xgb_params = {
    'eta': 0.3, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'reg:squarederror',
    'nthread': 8,
    
    'seed': 1,
    'verbosity': 1,
}

In [12]:
model = xgb.train(xgb_params, dtrain, num_boost_round=100)
watchlist = [(dtrain, 'train'), (dval, 'val')] 
y_pred = model.predict(dval)
mean_squared_error(y_val, y_pred,squared=False)

0.22897404244864047

In [13]:
xgb_params = {
    'eta': 0.1, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'reg:squarederror',
    'nthread': 8,
    
    'seed': 1,
    'verbosity': 1,
}

model = xgb.train(xgb_params, dtrain, num_boost_round=100)
watchlist = [(dtrain, 'train'), (dval, 'val')] 
y_pred = model.predict(dval)
mean_squared_error(y_val, y_pred,squared=False)

0.2323352139407306

eta = 0.3 gives a better RMSE value